1. Preprocessing and cleaning
2. Train,test split
3. BOW,TFIDF , word2vec
4. Train ML algorithm

In [18]:
import pandas as pd

data = pd.read_csv('all_kindle_review.csv')
data.head()

Unnamed: 0.1  Unnamed: 0        asin  helpful  rating  \
0             0       11539  B0033UV8HI  [8, 10]       3   
1             1        5957  B002HJV4DE   [1, 1]       5   
2             2        9146  B002ZG96I4   [0, 0]       3   
3             3        7038  B002QHWOEU   [1, 3]       3   
4             4        1776  B001A06VJ8   [0, 1]       4   

                                          reviewText   reviewTime  \
0  Jace Rankin may be short, but he's nothing to ...   09 2, 2010   
1  Great short read.  I didn't want to put it dow...   10 8, 2013   
2  I'll start by saying this is the first of four...  04 11, 2014   
3  Aggie is Angela Lansbury who carries pocketboo...   07 5, 2014   
4  I did not expect this type of book to be in li...  12 31, 2012   

       reviewerID  reviewerName                   summary  unixReviewTime  
0  A3HHXRELK8BHQG        Ridley  Entertaining But Average      1283385600  
1  A2RGNZ0TRF578I  Holly Butler   Terrific menage scenes!      1381190400  
2  A3S0H2HV6U1I7F       Merissa          Snapdragon Alley      1397174400  
3   AC4OQW3GZ919J    Cleargrace    very light murder cozy      1404518400  
4  A3C9V987IQHOQD      Rjostler                      Book      1356912000

In [19]:
df = data[['reviewText' , 'rating']]
df.head()

reviewText  rating
0  Jace Rankin may be short, but he's nothing to ...       3
1  Great short read.  I didn't want to put it dow...       5
2  I'll start by saying this is the first of four...       3
3  Aggie is Angela Lansbury who carries pocketboo...       3
4  I did not expect this type of book to be in li...       4

In [20]:
df.shape

(12000, 2)

In [21]:
df.isnull().sum()

reviewText    0
rating        0
dtype: int64

In [22]:
df['rating'].unique()

array([3, 5, 4, 2, 1])

In [23]:
df['rating'].value_counts()

rating
5    3000
4    3000
3    2000
2    2000
1    2000
Name: count, dtype: int64

In [24]:
##Preprocessing and Cleaning
##positive review is 1 and negative review is 0
df['rating'] = df['rating'].apply(lambda x:0 if x<3 else 1)


C:\Users\ritig\AppData\Local\Temp\ipykernel_24284\2595920031.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['rating'] = df['rating'].apply(lambda x:0 if x<3 else 1)


In [25]:
df['rating'].value_counts()

rating
1    8000
0    4000
Name: count, dtype: int64

In [26]:
##1. lower all the cases

df['reviewText']=df['reviewText'].str.lower()

C:\Users\ritig\AppData\Local\Temp\ipykernel_24284\4275719473.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['reviewText']=df['reviewText'].str.lower()


In [27]:
df.head()

reviewText  rating
0  jace rankin may be short, but he's nothing to ...       1
1  great short read.  i didn't want to put it dow...       1
2  i'll start by saying this is the first of four...       1
3  aggie is angela lansbury who carries pocketboo...       1
4  i did not expect this type of book to be in li...       1

In [28]:
##Clean the data
### removing special characters
import re

In [29]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ritig\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [30]:
import sys
!{sys.executable} -m pip install bs4
from bs4 import BeautifulSoup

In [32]:
!{sys.excutable} -m pip install lxml


'{sys.excutable}' is not recognized as an internal or external command,
operable program or batch file.


In [34]:

## Removing special characters
df['reviewText']=df['reviewText'].apply(lambda x:re.sub('[^a-z A-z 0-9-]+', '',x))
## Remove the stopswords
df['reviewText']=df['reviewText'].apply(lambda x:" ".join([y for y in x.split() if y not in stopwords.words('english')]))
## Remove url 
df['reviewText']=df['reviewText'].apply(lambda x: re.sub(r'(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', '' , str(x)))
## Remove html tags
df['reviewText']=df['reviewText'].apply(lambda x: BeautifulSoup(x, 'html.parser').get_text())
## Remove any additional spaces
df['reviewText']=df['reviewText'].apply(lambda x: " ".join(x.split()))

C:\Users\ritig\AppData\Local\Temp\ipykernel_24284\4033926759.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['reviewText']=df['reviewText'].apply(lambda x:re.sub('[^a-z A-z 0-9-]+', '',x))
C:\Users\ritig\AppData\Local\Temp\ipykernel_24284\4033926759.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['reviewText']=df['reviewText'].apply(lambda x:" ".join([y for y in x.split() if y not in stopwords.words('english')]))
C:\Users\ritig\AppData\Local\Temp\ipykernel_24284\4033926759.py:6: SettingWithCop

In [35]:
df.head()

reviewText  rating
0  jace rankin may short hes nothing mess man hau...       1
1  great short read didnt want put read one sitti...       1
2  ill start saying first four books wasnt expect...       1
3  aggie angela lansbury carries pocketbooks inst...       1
4  expect type book library pleased find price right       1

In [36]:
## Lemmatizer
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

In [37]:
def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

In [38]:
df['reviewText'] = df['reviewText'].apply(lambda x:lemmatize_words(x))

C:\Users\ritig\AppData\Local\Temp\ipykernel_24284\3509282327.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['reviewText'] = df['reviewText'].apply(lambda x:lemmatize_words(x))


In [39]:
##TrAIN TEST SPLIT
from sklearn.model_selection import train_test_split
x_train , x_test , y_train , y_test = train_test_split(df['reviewText'] , df['rating'],
                    test_size = 0.2)

In [40]:
from sklearn.feature_extraction.text import CountVectorizer
bow = CountVectorizer()

x_train_bow = bow.fit_transform(x_train).toarray()
x_test_bow = bow.transform(x_test).toarray()

In [41]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()

x_train_tfidf = tfidf.fit_transform(x_train).toarray()
x_test_tfidf = tfidf.transform(x_test).toarray()

In [42]:
from sklearn.naive_bayes import GaussianNB
nb_model_bow = GaussianNB().fit(x_train_bow , y_train)
nb_model_tfidf = GaussianNB().fit(x_train_tfidf,y_train)

In [43]:
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
y_pred_bow = nb_model_bow.predict(x_test_bow)
y_pred_tfidf = nb_model_tfidf.predict(x_test_tfidf)


print("BOW accuracy: ",accuracy_score(y_test , y_pred_bow))
print("tfidf accuracy: ",accuracy_score(y_test , y_pred_tfidf))






BOW accuracy:  0.585
tfidf accuracy:  0.5883333333333334


In [44]:
confusion_matrix(y_test , y_pred_bow)

array([[509, 270],
       [726, 895]])

In [45]:
confusion_matrix(y_test , y_pred_tfidf)

array([[497, 282],
       [706, 915]])